In [ ]:
! pip -qq install stable-audio-tools

In [ ]:
from huggingface_hub import notebook_login, Repository

# Login to Hugging Face
notebook_login()

### List of Famous Songs from 1904
* https://www.goldstandardsonglist.com/Pages_Sort_4a/Sort_4a_1900-1909.htm#1904%20(4%20Songs)

In [ ]:
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond

device = "cuda" if torch.cuda.is_available() else "cpu"

# Download model
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]

model = model.to(device)

# Set up text and timing conditioning
conditioning = [{
    "prompt": "60 BPM music like Entertainer, The by Joplin, Scott; REC: Scott Joplin; New Orleans Ragtime Orchestra; George Shearing in style of ragtime.",
    "seconds_start": 0, 
    "seconds_total": 30
}]

# Generate stereo audio
output = generate_diffusion_cond(
    model,
    steps=100,
    cfg_scale=7,
    conditioning=conditioning,
    sample_size=sample_size,
    sigma_min=0.3,
    sigma_max=500,
    sampler_type="dpmpp-3m-sde",
    device=device
)

# Rearrange audio batch to a single sequence
output = rearrange(output, "b d n -> d (b n)")

# Peak normalize, clip, convert to int16, and save to file
output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()


In [ ]:
torchaudio.save("output.wav", output, sample_rate)


In [ ]:
from IPython.display import Audio
sound_file = 'output.wav'
Audio(sound_file, autoplay=True)